### This method optimises the surface normal using dense ROI matching, with LM or gauss newton.

Check out https://www.ri.cmu.edu/pub_files/pub3/baker_simon_2002_3/baker_simon_2002_3.pdf for general outline of the algorithm. Requires initial norm.

In [1]:
import cv2
import numpy as np
import math
from scipy.sparse import csr_matrix


fx = 718.856
fy = 718.856
cx = 607.1928
cy = 185.2157

cam_mat = np.array([
        [fx, 0, cx],
        [0, fy, cy],
        [0,  0,  1]
    ])

In [2]:
def gx(R, T, n, K):
    Kinv = np.linalg.inv(K)
#     G = K.dot(R - (T.dot(n.transpose()))).dot(Kinv)
    G = K.dot(R + (T.dot(n.transpose()))).dot(Kinv)

    return G
    

def eval_jacobian_f(H, point_indices):
    h = H.flatten()
    
    Jf = np.zeros( (point_indices.shape[0] * 2, 9) )
    
    for row in range(point_indices.shape[0]):
        x = point_indices[row, 0]
        y = point_indices[row, 1]
        
        Jf[2 * row, 0] = x / (h[6] * x + h[7] * y + h[8])
        Jf[2 * row, 1] = y / (h[6] * x + h[7] * y + h[8])
        Jf[2 * row, 2] = 1. / (h[6] * x + h[7] * y + h[8] )
#         Jf[2 * row, 3] = 0
#         Jf[2 * row, 4] = 0
#         Jf[2 * row, 5] = 0
        n = h[0] * x + h[1] * y + h[2]
        d = (h[6] * x + h[7] * y + h[8])**2
        Jf[2 * row, 6] = -(n / d) * x
        Jf[2 * row, 7] = -(n / d) * y
        Jf[2 * row, 8] = -(n / d)
        
#         Jf[2 * row + 1, 0] = 0
#         Jf[2 * row + 1, 1] = 0
#         Jf[2 * row + 1, 2] = 0
        Jf[2 * row + 1, 3] = x / (h[6] * x + h[7] * y + h[8])
        Jf[2 * row + 1, 4] = y / (h[6] * x + h[7] * y + h[8])
        Jf[2 * row + 1, 5] = 1 / (h[6] * x + h[7] * y + h[8])
        n2 = h[3] * x + h[4] * y + h[5]
        Jf[2 * row + 1, 6] = -(n2 / d) * x
        Jf[2 * row + 1, 7] = -(n2 / d) * y
        Jf[2 * row + 1, 8] = -n2 /d
    #print("JF_slow")
    #print(Jf[:2])
    return Jf

def eval_jacobian_f_fast(H, point_indices):
    h = H.flatten()
    
    Jf = np.zeros( (point_indices.shape[0] * 2, 9) )
    
    xy1 = np.hstack((point_indices, np.ones((point_indices.shape[0], 1))) )
    h678 = h[-3:]
    h012 = h[:3]
    h345 = h[3:6]
    
    #print(xy1)
    
    d1 = (xy1 * h678).sum(axis=1).reshape((-1, 1))  #(h[6] * x + h[7] * y + h[8])
    
    #print(xy1.shape)
    #print(d1.shape)
    Jf[::2, :3] = xy1 / d1
    
    n = (h012 * xy1).sum(axis=1).reshape((-1, 1)) 
    n2 = (h345 * xy1).sum(axis=1).reshape((-1, 1)) 
    d = d1**2
    
    Jf[::2, -3:] = -(n/d) * xy1
    
    Jf[1::2, 3:6] = xy1 / d1
    Jf[1::2, -3:] = -(n2/d) * xy1 
    #print("JF_fast")
    #print(Jf[:2])
    return Jf
    
#     for row in range(point_indices.shape[0]):
#         x = point_indices[row, 0]
#         y = point_indices[row, 1]
        
#         Jf[2 * row, 0] = x / (h[6] * x + h[7] * y + h[8])
#         Jf[2 * row, 1] = y / (h[6] * x + h[7] * y + h[8])
#         Jf[2 * row, 2] = 1. / (h[6] * x + h[7] * y + h[8] )
# #         Jf[2 * row, 3] = 0
# #         Jf[2 * row, 4] = 0
# #         Jf[2 * row, 5] = 0
#         n = h[0] * x + h[1] * y + h[2]
#         d = (h[6] * x + h[7] * y + h[8])**2
#         Jf[2 * row, 6] = -(n / d) * x
#         Jf[2 * row, 7] = -(n / d) * y
#         Jf[2 * row, 8] = -(n / d)
        
# #         Jf[2 * row + 1, 0] = 0
# #         Jf[2 * row + 1, 1] = 0
# #         Jf[2 * row + 1, 2] = 0
#         Jf[2 * row + 1, 3] = x / (h[6] * x + h[7] * y + h[8])
#         Jf[2 * row + 1, 4] = y / (h[6] * x + h[7] * y + h[8])
#         Jf[2 * row + 1, 5] = 1 / (h[6] * x + h[7] * y + h[8])
#         n2 = h[3] * x + h[4] * y + h[5]
#         Jf[2 * row + 1, 6] = -(n2 / d) * x
#         Jf[2 * row + 1, 7] = -(n2 / d) * y
#         Jf[2 * row + 1, 8] = -n2 /d
    
#     return Jf
         
def eval_jacobian_I(grad_x, grad_y, transformed_indices):
    npoints = transformed_indices.shape[0]
    
    rows = []
    cols = []
    data = []
    
#     delta_xy = transformed_indices - point_indices
    #print(delta_xy)
    transformed_indices2 = np.around(transformed_indices).astype(int)
    for row in range(npoints):
        x, y = transformed_indices2[row]
#        x = int(x)
#        y = int(y)
#         xdiff, ydiff = delta_xy[row]
        rows.append(row)
        rows.append(row)
        cols.append(2*row)
        cols.append(2 * row + 1)
        data.append(grad_x[y, x])
        data.append(grad_y[y, x])        
    
    JI = csr_matrix((data, (rows, cols)) , shape = (npoints, npoints * 2),  dtype='float')
         
    return JI

def eval_jacobian_I_fast(grad_x, grad_y, transformed_indices):
    npoints = transformed_indices.shape[0]
    transformed_indices2 = np.around(transformed_indices).astype(int)

    rows = np.repeat(np.arange(npoints), 2)
    cols = np.arange(npoints * 2)
    
    gx = grad_x[transformed_indices2[:, 1], transformed_indices2[:, 0]].reshape((-1, 1))
    gy = grad_y[transformed_indices2[:, 1], transformed_indices2[:, 0]].reshape((-1, 1))
    
    data = np.hstack((gx, gy)).flatten()
    JI = csr_matrix((data, (rows, cols)) , shape = (npoints, npoints * 2),  dtype='float')

    return JI
    

def eval_jacobian_g(R, t, n1, n2, n3, cam_mat):
    f1 = cam_mat[0, 0]
    f2 = cam_mat[1, 1]
    c1 = cam_mat[0, 2]
    c2 = cam_mat[1, 2]
    
    Kinv = np.linalg.inv(cam_mat)
    fi1 = Kinv[0 ,0]
    fi2 = Kinv[1, 1]
    ci1  = Kinv[0, 2]
    ci2 =  Kinv[1, 2]
    
    J = np.zeros( (9, 3) )
    
    x = t[0, 0]
    y = t[1, 0]
    z = t[2, 0]
    
#     J[0, 0] = f1 * fi1 * (-x) + c1 * fi1 * (-z)
    J[0, 0] = f1 * fi1 * (x) + c1 * fi1 * (z)
    J[0, 1] = 0
    J[0, 2] = 0
    
    J[1, 0] = 0
    J[1, 1] = f1 * fi2 * (x) + c1 * fi2 * (z)
#     J[1, 1] = f1 * fi2 * (-x) + c1 * fi2 * (-z)
    J[1, 2] = 0
    
#     J[2, 0] = f1 * (-ci1 * x) + c1 * (-ci1 * z) 
    J[2, 0] = f1 * (ci1 * x) + c1 * (ci1 * z) 
#     J[2, 1] = f1 * (-ci2 * x) + c1 * (-ci2 * z) 
    J[2, 1] = f1 * (ci2 * x) + c1 * (ci2 * z) 
    J[2, 2] = f1 * (x) + c1 * (z)
#     J[2, 2] = f1 * (-x) + c1 * (-z)
    
#    J[3, 0] = f2 * fi1 * (-y) + c2 * fi1 * (-z)
    J[3, 0] = f2 * fi1 * (y) + c2 * fi1 * (z)
    J[3, 1] = 0
    J[3, 2] = 0
    
    J[4, 0] = 0
    J[4, 1] = f2 * fi2 * (y) + c2 * fi2 * (z)
#     J[4, 1] = f2 * fi2 * (-y) + c2 * fi2 * (-z)
    J[4, 2] = 0
    
#    J[5, 0] = f2 * ci1 * (-y) + c2 * ci1 * (-z) 
    J[5, 0] = f2 * ci1 * (y) + c2 * ci1 * (z) 
#     J[5, 1] = f2 * ci2 * (-y) + c2 * ci2 * (-z)
    J[5, 1] = f2 * ci2 * (y) + c2 * ci2 * (z)
    J[5, 2] = f2 * (y) + c2 * (z)
#     J[5, 2] = f2 * (-y) + c2 * (-z)
    
#    J[6, 0] = fi1 * (-z)
    J[6, 0] = fi1 * (z)
    J[6, 1] = 0
    J[6, 2] = 0
    
#     J[7, 0] = fi2 * (-z)
    J[7, 0] = 0 #fi2 * (z)
    J[7, 1] = fi2 * z 
    J[7, 2] = 0 
    
#     J[8, 0] = ci1 * (-z)
#     J[8, 1] = ci2 * (-z)
#     J[8, 2] = -z
    J[8, 0] = ci1 * (z)
    J[8, 1] = ci2 * (z)
    J[8, 2] = z
    
    return J
    
def eval_jacobian(R, t, n1, n2, n3, cam_mat, point_indices, gradientx, gradienty, transformed_indices):
    n = np.array([[n1], [n2], [n3]])
    H = gx(R, t, n, cam_mat)
    
    Jf = eval_jacobian_f(H, point_indices)
    Jg = eval_jacobian_g(R, t, n1, n2, n3, cam_mat)
    JI = eval_jacobian_I(gradientx, gradienty, transformed_indices)
    J = JI.dot(Jf.dot(Jg))
    
    #print("Jg: \n{}".format(Jg) )
    #print("Jf: \n{}".format(Jf) )
    #print("JI: \n{}".format(JI) )

    return J

def eval_jacobian_fast(R, t, n1, n2, n3, cam_mat, point_indices, gradientx, gradienty, transformed_indices):
    n = np.array([[n1], [n2], [n3]])
    H = gx(R, t, n, cam_mat)
    
    Jf = eval_jacobian_f_fast(H, point_indices)
    Jg = eval_jacobian_g(R, t, n1, n2, n3, cam_mat)
    JI2 = eval_jacobian_I_fast(gradientx, gradienty, transformed_indices)
    J = JI2.dot(Jf.dot(Jg))
    
    #print("Jg: \n{}".format(Jg) )
    #print("Jf: \n{}".format(Jf) )
    #print("JI: \n{}".format(JI) )

    return J


def eval_objective(R, t, n1, n2, n3, cam_mat, point_indices, img) :
    #assert observations.shape[0] == point_indices.shape[0]
    
    n = np.array([[n1], [n2], [n3]])
    
    H = (gx(R, t, n, cam_mat))
    
    uv = np.ones( (3, point_indices.shape[0]))
    uv[:2, :] = point_indices.transpose()
    
    target_uv = H.dot(uv)
    target_uv /= target_uv[2, :]
    target_uv = np.around(target_uv).astype(int)
    
    intensities = img[target_uv[1, :], target_uv[0, :]]
    return intensities
    
    
def get_image_gradient(img):
    gx = cv2.Scharr(img, cv2.CV_64F, 1, 0)
    gy = cv2.Scharr(img, cv2.CV_64F, 0, 1)
    
    #gx = cv2.Sobel(img,cv2.CV_64F,1,0,ksize=3)
    #gy = cv2.Sobel(img,cv2.CV_64F,0,1,ksize=3)
    return gx, gy

def get_observations(img, point_indices):
    return img[point_indices[:,1], point_indices[:,0]]

def get_point_indices(ROI, img_dim):
    h, w = img_dim
    point_indices = []
    for r in range(h):
        for c in range(w):
            p = (c, r)
            if cv2.pointPolygonTest(ROI, p, 0) >= 0:
                point_indices.append([c, r])
    
    point_indices = np.array(point_indices)
    point_indices = point_indices#[::2]
    return point_indices

def get_valid_indices(R, t, n1, n2, n3, cam_mat, point_indices, img_dims):

    n = np.array([[n1], [n2], [n3]])    
    H = gx(R, t, n, cam_mat)
#     Hinv = np.linalg.inv(H)
    
    uv = np.ones( (3, point_indices.shape[0]))
    uv[:2, :] = point_indices.transpose()
    
#     print("before: {} ".format(point_indices[100]) )
        
    target_uv = H.dot(uv)
    target_uv /= target_uv[2, :]
    #target_uv = np.around(target_uv).astype(int)
    
#     reverse_uv = Hinv.dot(uv)
#     reverse_uv /= reverse_uv[2, :]
        
#     print("after: {} ".format(target_uv[:, 100]) )

    row_indices_valid = target_uv[1, :] < img_dims[0]
    #print(H.shape)
    #print(uv.shape)
    #print(target_uv.shape)
    #print("ello {}".format(target_uv[1, :].shape))
    
    valid_target = target_uv.transpose()[:, :2]#[row_indices_valid]
    
    return row_indices_valid, valid_target
    
    
#def lm(r, p, h, x, y, z, n1, n2, n3, lamb, max_iters):
def lm(R, t, n1, n2, n3, lamb, max_iters, point_indices, K, src_img, target_img):
    
    print("current scale: {}".format(1.7 * np.linalg.norm(np.array([[n1], [n2], [n3]]))))

#    grad_x, grad_y = get_image_gradient(src_img)
    grad_x, grad_y = get_image_gradient(target_img)
#     observations = get_observations(target_img, point_indices)

    last_error = 1000000
    last_delta = [0., 0., 0.]
    
        
    for r in range(max_iters):
        valids, transformed_indices = get_valid_indices(R, t, n1, n2, n3, K, point_indices, src_img.shape)
#         print(transformed_indices.shape)
#         print(transformed_indices)
        #print(valids.shape)
        #print(point_indices.shape)
        cur_indices = point_indices[valids]
        
        expected_observations = get_observations(target_img, np.around(transformed_indices).astype(int)).astype('float')
        observations = get_observations(src_img, cur_indices).astype('float')

        
        #expected_observations = eval_objective(R, t, n1, n2, n3, K, cur_indices, target_img).astype('float')
#         J = eval_jacobian(R, t, n1, n2, n3, K, cur_indices, grad_x, grad_y, transformed_indices)
        J = eval_jacobian_fast(R, t, n1, n2, n3, K, cur_indices, grad_x, grad_y, transformed_indices)
        JT = J.transpose()
        
        #print("J: {}".format(J) )
        #print("J2: {}".format(J2) )
        
        #assert(np.array_equal(J, J2))
        
        JJ = JT.dot(J)
                                       
        #print(observations.dtype)
        #print(expected_observations.dtype)
        delta = np.linalg.inv( JJ + lamb * np.diag(np.diag(JJ)) ).dot(JT.dot(observations - expected_observations) )

        n1 += delta[0]
        n2 += delta[1]
        n3 += delta[2]
                
        err = observations - expected_observations
        err = (err * err).sum()
        
        if err <= last_error:
            lamb /= 3.
            lamb = max(0.00001, lamb)
        else:
            lamb *= 2.
            lamb = min(50, lamb)
            
        last_error = err
        
        #print(np.array([[n1], [n2], [n3]]))
        print("iter {} err: {} current scale: {} lambda: {} points: {}".format(
            r, err, 1.7 * np.linalg.norm(np.array([[n1], [n2], [n3]])), lamb, cur_indices.shape[0]))
        
        
    

In [12]:
import os

#for dataset 00-02
fx = 718.856
fy = 718.856
cx = 607.1928
cy = 185.2157

cam_mat = np.array([
        [fx, 0, cx],
        [0, fy, cy],
        [0,  0,  1]
    ])
actual_height = 1.7

seq_str = '00'
img_data_dir = '/home/cy/Documents/data/kitti/data_odometry_gray/dataset/sequences/{}/image_0/'.format(seq_str)
gt_file = '/home/cy/Documents/data/kitti/data_odometry_poses/{}.txt'.format(seq_str)

nframes = len([f for f in os.listdir(img_data_dir) if f.endswith(".png")])

seq = int(seq_str)

sample_idx = 900
fn = str(sample_idx).rjust(6, '0')
fnm1 = str(sample_idx+1).rjust(6, '0')
img1= cv2.imread(os.path.join(img_data_dir,"{}.png".format(fn)), 0).astype('float')
img2= cv2.imread(os.path.join(img_data_dir,"{}.png".format(fnm1)), 0).astype('float')
img1 /= 255.
img2 /= 255.

# [[-0.03108007]
#  [ 0.99932888]
#  [ 0.01938621]]
# array([[-0.01456786],
#        [ 0.46840554],
#        [ 0.00908671]])
# 3.3931844612668813
# scale using roi: 0.797
# [[0.0636999 ]
#  [0.99793658]
#  [0.00805662]]
# array([[0.03097199],
#        [0.48521395],
#        [0.00391727]])
# 5.082832637658015
# scale using roi+geo constraints: 0.827
# ground truth scale: 0.8307917549542936
# R:
# array([[ 9.99993673e-01, -3.47749990e-03,  7.48896327e-04],
#        [ 3.47806029e-03,  9.99993672e-01, -7.48286156e-04],
#        [-7.46289423e-04,  7.50886129e-04,  9.99999440e-01]])
# T:
# array([[ 0.02089309],
#        [ 0.04358919],
#        [-0.99883105]])


R = np.array([[ 9.99993673e-01, -3.47749990e-03,  7.48896327e-04],
       [ 3.47806029e-03,  9.99993672e-01, -7.48286156e-04],
       [-7.46289423e-04,  7.50886129e-04,  9.99999440e-01]])

t = np.array([[ 0.02089309],
       [ 0.04358919],
       [-0.99883105]])

n2 = np.array([[0.03097199],
       [0.48521395],
       [0.00391727]])

ROI = np.array([
    [500, 230], [700, 230], [800, 330], [400, 330]
])
ROI = ROI.reshape((-1, 1, 2))

point_indices = get_point_indices(ROI, img1.shape)

#lm(R, t, n[0,0], n[1,0], n[2,0], 1, 500, point_indices, cam_mat, img1, img2)

In [13]:
lm(R, t, n2[0,0], n2[1,0], n2[2,0], 0, 50, point_indices, cam_mat, img1, img2)

current scale: 0.826569276480484
iter 0 err: 505.2450903498654 current scale: 0.8271820009410724 lambda: 1e-05 points: 30401
iter 1 err: 498.58409842368326 current scale: 0.8277196692170503 lambda: 1e-05 points: 30401
iter 2 err: 492.25307189542485 current scale: 0.8281587605907915 lambda: 1e-05 points: 30401
iter 3 err: 487.4925336409073 current scale: 0.8285735566478241 lambda: 1e-05 points: 30401
iter 4 err: 482.74638985005765 current scale: 0.8289640183153452 lambda: 1e-05 points: 30401
iter 5 err: 479.99849288735106 current scale: 0.8292559529075578 lambda: 1e-05 points: 30401
iter 6 err: 475.3145405613226 current scale: 0.8295260032136945 lambda: 1e-05 points: 30401
iter 7 err: 471.0793694732795 current scale: 0.8298943949834916 lambda: 1e-05 points: 30401
iter 8 err: 466.913417916186 current scale: 0.8303596709781005 lambda: 1e-05 points: 30401
iter 9 err: 463.44607458669736 current scale: 0.830969634968663 lambda: 1e-05 points: 30401
iter 10 err: 458.5667358708189 current scale

iter 4 err: 482.74638985005765 current scale: 0.8289640183153452 lambda: 1e-05 points: 30401
iter 5 err: 479.99849288735106 current scale: 0.8292559529075578 lambda: 1e-05 points: 30401
iter 6 err: 475.3145405613226 current scale: 0.8295260032136945 lambda: 1e-05 points: 30401
iter 7 err: 471.0793694732795 current scale: 0.8298943949834916 lambda: 1e-05 points: 30401
iter 8 err: 466.913417916186 current scale: 0.8303596709781005 lambda: 1e-05 points: 30401
iter 9 err: 463.44607458669736 current scale: 0.830969634968663 lambda: 1e-05 points: 30401
iter 10 err: 458.5667358708189 current scale: 0.8318691694350735 lambda: 1e-05 points: 30401
iter 11 err: 453.1893425605536 current scale: 0.8328128674269041 lambda: 1e-05 points: 30401
iter 12 err: 447.3437600922722 current scale: 0.8337892070080389 lambda: 1e-05 points: 30401
iter 13 err: 442.7457747020377 current scale: 0.8347416944573616 lambda: 1e-05 points: 30401
iter 14 err: 439.07410995770863 current scale: 0.8356724118019621 lambda: 1

iter 46 err: 263.0294194540561 current scale: 0.8329903986513512 lambda: 1e-05 points: 30401
iter 47 err: 261.355940023068 current scale: 0.8319255762106893 lambda: 1e-05 points: 30401
iter 48 err: 256.9830219146482 current scale: 0.8309449695929432 lambda: 1e-05 points: 30401
iter 49 err: 252.92725874663589 current scale: 0.8299891232941274 lambda: 1e-05 points: 30401
current scale: 0.826569276480484
iter 0 err: 505.2450903498654 current scale: 0.8271820009410724 lambda: 1e-05 points: 30401
iter 1 err: 498.58409842368326 current scale: 0.8277196692170503 lambda: 1e-05 points: 30401
iter 2 err: 492.25307189542485 current scale: 0.8281587605907915 lambda: 1e-05 points: 30401
iter 3 err: 487.4925336409073 current scale: 0.8285735566478241 lambda: 1e-05 points: 30401
iter 4 err: 482.74638985005765 current scale: 0.8289640183153452 lambda: 1e-05 points: 30401
iter 5 err: 479.99849288735106 current scale: 0.8292559529075578 lambda: 1e-05 points: 30401
iter 6 err: 475.3145405613226 current s

iter 49 err: 252.92725874663589 current scale: 0.8299891232941274 lambda: 1e-05 points: 30401
current scale: 0.826569276480484
iter 0 err: 505.2450903498654 current scale: 0.8271820009410724 lambda: 1e-05 points: 30401
iter 1 err: 498.58409842368326 current scale: 0.8277196692170503 lambda: 1e-05 points: 30401
iter 2 err: 492.25307189542485 current scale: 0.8281587605907915 lambda: 1e-05 points: 30401
iter 3 err: 487.4925336409073 current scale: 0.8285735566478241 lambda: 1e-05 points: 30401
iter 4 err: 482.74638985005765 current scale: 0.8289640183153452 lambda: 1e-05 points: 30401
iter 5 err: 479.99849288735106 current scale: 0.8292559529075578 lambda: 1e-05 points: 30401
iter 6 err: 475.3145405613226 current scale: 0.8295260032136945 lambda: 1e-05 points: 30401
iter 7 err: 471.0793694732795 current scale: 0.8298943949834916 lambda: 1e-05 points: 30401
iter 8 err: 466.913417916186 current scale: 0.8303596709781005 lambda: 1e-05 points: 30401
iter 9 err: 463.44607458669736 current sca